# Проект "Music of capitals"

# Описание проекта

В этом проекте предстоит сравнить музыкальные вкусы жителей двух городов Санкт-Петербурга и Москвы. Отличаются ли музыкальные предпочтения культурной столицы от вечно работающего мегаполиса?

# Цели исследования

Проверить разработанные гипотезы:
- Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному;
- В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города;
- Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

# Задачи исследования

- Обзор данных;
- Предобработка данных;
- Проверка гипотез.

# Исходные данные

Таблица "YMusic"

* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

## Обзор данных

Импорт библиотеки `pandas`:

In [9]:
# импорт библиотеки
import pandas as pd

Чтение рабочего файла `/datasets/music_of_capitals/music of capitals.csv` и сохранение его в переменной `df`:

In [11]:
# чтение файла с данными и сохранение в  переменной df
df = pd.read_csv('C:\Dev\Jupyter\dist\projects\datasets\music_of_capitals\music of capitals.csv')

Выведение на экран первых десяти строк таблицы:

In [12]:
# получение первых 10 строк таблицы df
display(df.head(10))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Получение общей информации о таблице:

In [9]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Стоит обратить внимание на исходные названия столбцов в таблице:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок можем выделить три нарушения стиля:
1. Строчные буквы сочетаются с прописными;
2. Встречаются пробелы;
3. При переходе на "snake_case" название идентификатора пользователя стоит привести в вид "user_id".

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Изменение стиля заголовков
Выведем на экран названия столбцов:

In [8]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем:
* к названию столбцов применим "snake_case";
* все символы сделаем строчными;
* устраним пробелы.

Изменения названий столбцов должны выглядеть вот так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [9]:
# переименуем столбцы, используя "snake_case"
df = df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})# переименование столбцов

In [10]:
# проверка результата
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Подсчет, устранение, замена пропущенных значений в данных

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски достаточно заменить обозначениями `unknown`.

Пропуски в `genre` могут помешать сравнению музыкальных вкусов в столицах. В этом проекте отсутствует возможность установить причину пропусков и восстановить данные. Тем не менее, мы заполним и эти пропуски обозначениями `unknown`.

In [11]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [15]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for col in columns_to_replace:
    df[col] = df[col].fillna('unknown')

In [17]:
# убедимся, что пропусков в данных нет
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Подсчет и удаление дубликатов из данных

In [18]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [19]:
# удаление дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index()

In [20]:
# проверка на отсутствие явных дубликатов
df.duplicated().sum()

0

Избавимся от неявных дубликатов. В столбце `genre` название одного и того же жанра может быть записано по-разному. Это может повлиять на результат исследования.

In [23]:
# просмотр уникальных названий жанров с сортировкой в алфавитном порядке
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Стоит обратить внимание на неявные дубликаты жанра `hiphop`. Названия с ошибками, альтернативные названия жанра. Очистим от них таблицу.

Напишем функцию `del_wrong_genres()` с двумя параметрами: 
* `uncorr_genres` — список дубликатов,
* `corr_genre` — строка с правильным значением.

Функция исправит значения в  колонке `genre` в таблице `df`: заменит каждое значение из списка `uncorr_genres` на значение из `corr_genre`.

In [29]:
# функция для замены неявных дубликатов
def del_wrong_genres(uncorr_genres,corr_genre):
    for value in uncorr_genres:
        df['genre'] = df['genre'].replace(uncorr_genres,corr_genre)
        

Теперь вызовем `del_wrong_genres()` и передадим ей аргументы, чтобы устранить неявные дубликаты. Вместо `hip`, `hop` и `hip-hop` укажем `hiphop`.

In [30]:
# устранение неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop']
right_name = 'hiphop'
del_wrong_genres(duplicates, right_name)

In [31]:
# проверка на неявные дубликаты
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'tatar', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'shanson', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'contemporary', 'new', 'soul', 'holiday', 'german',
       'tropical', 'fairytail', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'chanson', 'numetal', 'vocal', 'estrada', 'russian',
       'classicmetal', 'dubstep', 'club', 'deep', 'southern', 'black',
       'folkrock', 'fitness'

**Вывод**

Выявленные на этапе предобработки данных проблемы с нарушением в стиле заголовков, пропущенными значениями, наличием дубликатов были решены. Заголовки исправлены, явные и неявные дубликаты удалены, пропущенные значения заменены на `unknown`.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза: пользователи по-разному слушают музыку в Москве и Санкт-Петербурге по понедельникам, средам и пятницам.

Для проверки гипотезы:
* Разделим пользователей Москвы и Санкт-Петербурга;
* Сравним количество треков  прослушанных каждой группой пользователей в указанные дни.

In [32]:
# группировка данных по городу и подсчёт количества прослушиваний в каждом
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

Стоит отметить, что в Москве прослушиваний больше, чем в СПб. Это может говорить о том, что количество пользователей из Москвы превалирует.

In [33]:
# группировка данных по дням и подсчёт количества прослушиваний в каждом
df.groupby('day')['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

### Функция для сравнение поведения пользователей двух столиц

Напишем функцию, которая объединит расчёты с группировкой по дням и городам. Применим последовательную фильтрацию с логической индексацией. Подсчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную.

In [34]:
# Объявим функцию с двумя параметрами: day и city.
# В переменной track_list сохраним те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city.
# В переменной track_count сохраним число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
# Функция возвращает число - значение track_count.
def dc_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city' ]== city)]
    track_count = track_list['user_id'].count()
    return track_count

Применим `dc_tracks()`,меняя значение параметров так, чтобы получить данные для каждого города в каждый из трёх дней.

In [42]:
# количество прослушиваний по понедельникам
print('Количество прослушиваний по понедельникам:')
print('')
print('Москва:',dc_tracks('Monday','Moscow'))
print('')
print('Санкт-Петербург:',dc_tracks('Monday','Saint-Petersburg'))

Количество прослушиваний по понедельникам:

Москва: 15740

Санкт-Петербург: 5614


In [45]:
# количество прослушиваний по средам
print('Количество прослушиваний по средам:')
print('')
print('Москва:',dc_tracks('Wednesday','Moscow'))
print('')
print('Санкт-Петербург:',dc_tracks('Wednesday','Saint-Petersburg'))

Количество прослушиваний по средам:

Москва: 11056

Санкт-Петербург: 7003


In [46]:
# количество прослушиваний по пятницам
print('Количество прослушиваний по пятницам:')
print('')
print('Москва:',dc_tracks('Friday','Moscow'))
print('')
print('Санкт-Петербург:',dc_tracks('Friday','Saint-Petersburg'))

Количество прослушиваний по пятницам:

Москва: 15945

Санкт-Петербург: 5895


Создадим c помощью `pd.DataFrame` таблицу, где:

* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;

* данные — результаты, которые вы получили с помощью `dc_tracks`.

In [57]:
# таблица с результатами
data = [['Moscow', 15740, 11056, 15945],
       ['Saint-Petersburg', 5614, 7003, 5895]]
columns = ['city', 'monday', 'wednesday', 'friday']

track_time_count = pd.DataFrame(data=data, columns=columns)
track_time_count

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Вывод**

По результатам проведенного исследования данных на предмет количества прослушиваний по понедельникам, средам и пятницам вв столицах заключим:
- В Москве пик прослушиваний приходится на понедельник и пятницу;
- В Петербурге пик приходится на среду.

Данные говорят в пользу первой гипотезы.

### Преобладающие жанры музывки в начале и в конце недели

Вторая гипотеза: утром в понедельник и вечером в пятницу в Москве преобладают одни жанры, а в Петербурге — другие.

In [58]:
# получение таблицы moscow_general из тех строк таблицы df, для которых значение в столбце 'city' равно 'Moscow'
moscow_genre = df[df['city']=='Moscow']

In [59]:
# получение таблицы spb_general из тех строк таблицы df, для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_genre = df[df['city']=='Saint-Petersburg']

Создадим функцию `genre_weekday()`. Она вернет информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [60]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используйте последовательную фильтрацию с помощью логической индексации.
# 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
#    столбцов и посчитать методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записать в переменную
#    genre_df_count
# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
#    в переменную genre_df_sorted
# 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)
def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day']==day) & (table['time']>time1) & (table['time']<time2)]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 12:00) и в пятницу вечером (с 17:00 до 24:00):

In [76]:
print('Москва, утро понедельника:')
print('')
print(genre_weekday(moscow_genre, 'Monday', '07:00', '12:00'))
print('')
print('Санкт-Петербург, утро понедельника:')
print('')
print(genre_weekday(spb_genre, 'Monday', '07:00', '12:00'))

Москва, утро понедельника:

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

Санкт-Петербург, утро понедельника:

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64


In [77]:
print('Москва, вечер пятницы:')
print('')
print(genre_weekday(moscow_genre, 'Friday', '17:00', '24:00'))
print('')
print('Санкт-Петербург, вечер пятницы:')
print('')
print(genre_weekday(spb_genre, 'Friday', '17:00', '24:00'))

Москва, вечер пятницы:

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

Санкт-Петербург, вечер пятницы:

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64


**Выводы**

Сравнив количество прослушиваний топ-10 жанров в начале и в конце недели отметим, что:

1. В Москве и Петербурге слушают похожую музыку. Отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

3. Вечер пятницы не меняет картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

4. Пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

Вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале и в конце недели;
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

### Жанровые предпочтения в Москве и Петербурге

Третья гипотеза: в Петербурге рэп слушают чаще, чем в Москве. В Москве преобладает поп-музыка.

Сгруппируем таблицу `moscow_genre` по жанру и посчитаем количество прослушиваний треков каждого жанра методом `count()`. Отсортируем результат в порядке убывания и сохраним его в таблице `all_moscow_genres`.

In [80]:
# cгруппируем таблицу moscow_genre по жанру и посчитаем количество прослушиваний треков каждого жанра методом count(). 
# Отсортируем результат в порядке убывания и сохраним его в таблице all_moscow_genres.
m_g_series = moscow_genre.groupby('genre')['genre'].count()
all_moscow_genres = m_g_series.sort_values(ascending=False)
print('Таблица жанровых предпочтений. Москва')
all_moscow_genres.head(10)

Таблица жанровых предпочтений. Москва


genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь для Петербурга.

In [82]:
s_pb_series = spb_genre.groupby('genre')['genre'].count()
all_spb_genres = s_pb_series.sort_values(ascending=False)
print('Таблица жанровых предпочтений. Санкт-Петербург')
all_spb_genres.head(10)

Таблица жанровых предпочтений. Санкт-Петербург


genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Вывод**

Гипотеза подтвердилась частично:
* Поп-музыка — самый популярный жанр в Москве. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Рэп одинаково популярен в Москве и Петербурге. 

## Итоги исследования

По результатам проверки трёх гипотез удалось установить:

1. День недели по-разному влияет на активность пользователей в Москве и Санкт-Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели независимо от города. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”;
* в Петербурге — джаз и классику.

Вторая гипотеза подтвердилась частично. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего. Предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза подтвердилась частично. Поп-музыка — самый популярный жанр в Москве. Рэп в СПб, как и в Москве, находится на дне ТОП-10 жанров.